<a href="https://colab.research.google.com/github/shahdcode/Credit-Card-Approval-Prediction/blob/main/DT_and_MLP_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import os

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from google.colab import files
from google.colab import drive
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
train_df = pd.read_csv('/content/drive/MyDrive/Ai project/train_scaled.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Ai project/validation_scaled.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Ai project/test_scaled.csv')


# Separate features and target
X_train = train_df.drop(columns=['status_mapped'])
y_train = train_df['status_mapped']

X_val = val_df.drop(columns=['status_mapped'])
y_val = val_df['status_mapped']

X_test = test_df.drop(columns=['status_mapped'])
y_test = test_df['status_mapped']


# Decision Tree Model with Grid Search

In [6]:
# Define the parameter grid for tuning
param_grid = {
    'criterion': ['gini', 'entropy'],            # Quality of split
    'max_depth': [3, 5, 7, None],                # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],            # Minimum samples required to split
    'min_samples_leaf': [1, 3, 5],              # Minimum samples in a leaf node
    'max_features': [None, 'sqrt', 'log2']      # Features considered for the best split
}

# Create the Decision Tree classifier
dt_model = DecisionTreeClassifier(random_state=42)

# Apply Grid Search
grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Get the best parameters and train final model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Validate and test the best model
train_score = accuracy_score(y_train, best_model.predict(X_train))
val_score = accuracy_score(y_val, best_model.predict(X_val))
test_score = accuracy_score(y_test, best_model.predict(X_test))

print(f"Best Parameters: {best_params}")
print(f"Training Accuracy: {train_score}")
print(f"Validation Accuracy: {val_score}")
print(f"Testing Accuracy: {test_score}")

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Parameters: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 2}
Training Accuracy: 0.9503152473137377
Validation Accuracy: 0.9500621632822213
Testing Accuracy: 0.9498653407913819


# MPL Model

In [30]:
X_train = tf.keras.utils.normalize(X_train , axis =1)
x_val = tf.keras.utils.normalize(X_val, axis=1)
x_test = tf.keras.utils.normalize(X_test, axis=1)

# Define the MLP model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128, activation='sigmoid', input_shape=(X_train.shape[1],)))  # Input layer
model.add(tf.keras.layers.Dense(64, activation='sigmoid'))                                   # Hidden layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))                                   # Output layer (multi-class)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=3)




Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


704/704 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8905 - loss: 0.4068 - val_accuracy: 0.9499 - val_loss: 0.2007
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9503 - loss: 0.1995 - val_accuracy: 0.9499 - val_loss: 0.1995
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9515 - loss: 0.1953 - val_accuracy: 0.9499 - val_loss: 0.1999


In [33]:
# Evaluate the model
model.save('mlp.keras')
model = tf.keras.models.load_model('mlp.keras')
loss, accuracy = model.evaluate(x_test,y_test)

print(f"Test Accuracy: {loss:.2f}")
print(f"Test Accuracy: {accuracy:.2f}")

151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9506 - loss: 0.1983
Test Accuracy: 0.20
Test Accuracy: 0.95
